In [5]:
from math import *

# convert minutiae template from txt format to iso19794-2-2005 format.


def to_iso19794(txtpath, isopath):
    width, height = 500, 610
    b_array = bytearray()
    with open(txtpath, "r") as textfile:
        lines = textfile.readlines()
    minutiae_num = len(lines)
    totalbytes = minutiae_num * 6 + 28 + 2

    b_array = bytearray(b"FMR\x00 20\x00")
    b_array += totalbytes.to_bytes(4, "big")
    b_array += bytearray(b"\x00\x00")
    b_array += width.to_bytes(2, "big")
    b_array += height.to_bytes(2, "big")
    b_array += bytearray(b"\x00\xc5\x00\xc5\x01\x00\x00\x00d")
    b_array += minutiae_num.to_bytes(1, "big")
    byte_list = [0, 0, 0, 0, 0, 0]
    for line in lines:
        line = line.split(" ")
        x, y, angle, min_type, quality = (
            int(float(line[0])),
            int(float(line[1])),
            int(float(line[2])),
            int(float(line[3])),
            int(float(line[4].strip())),
        )
        byte_list[1] = x % 256
        byte_list[0] = x // 256 + min_type * 64
        byte_list[2] = y // 256
        byte_list[3] = y % 256
        byte_list[4] = round((360 - angle) / 360 * 256) % 256
        byte_list[5] = quality
        b_array += bytearray(byte_list)

    zero = 0
    b_array += zero.to_bytes(2, "big")

    with open(isopath, "wb") as istfile:
        istfile.write(b_array)


import os

def main():
    txt_folder = "temp_ashu"
    iso_folder = "isoTemplate"

    # Create the output directory if it doesn't exist
    os.makedirs(iso_folder, exist_ok=True)

    for txt_file in os.listdir(txt_folder):
        if txt_file.endswith(".txt"):
            txt_path = os.path.join(txt_folder, txt_file)
            iso_file = os.path.splitext(txt_file)[0] + ".ist"
            iso_path = os.path.join(iso_folder, iso_file)
            to_iso19794(txt_path, iso_path)
            print(f"Conversion completed for: {txt_file} -> {iso_file}")

if __name__ == "__main__":
    main()



Conversion completed for: 12_9.txt -> 12_9.ist


In [13]:
# from collections import namedtuple

# # This code is to decode the iso19794-2-2005 and iso-19794-2-2011 templates of fingerprint minutiae.
# # VeriFinger SDK could recognize both of these two formats.

# Minutiae = namedtuple(
#     "Minutiae",
#     ["type", "x", "y", "orientation", "quality", "label"],
#     defaults=[None, None],
# )


# def load_iso19794(path, format):
#     if format == "19794-2-2005":
#         with open(path, "rb") as f:
#             t = f.read()
#         magic = int.from_bytes(t[0:4], "big")
#         version = int.from_bytes(t[4:8], "big")
#         total_bytes = int.from_bytes(t[8:12], "big")
#         im_w = int.from_bytes(t[14:16], "big")
#         im_h = int.from_bytes(t[16:18], "big")
#         resolution_x = int.from_bytes(t[18:20], "big")
#         resolution_y = int.from_bytes(t[20:22], "big")
#         f_count = int.from_bytes(t[22:23], "big")
#         reserved_byte = int.from_bytes(t[23:24], "big")
#         fingerprint_quality = int.from_bytes(t[26:27], "big")
#         minutiae_num = int.from_bytes(t[27:28], "big")
#         minutiaes = []
#         for i in range(minutiae_num):
#             x = 28 + 6 * i
#             min_type = (t[x] >> 6) & 0x3
#             min_x = int.from_bytes([t[x] & 0x3F, t[x + 1]], "big")
#             min_y = int.from_bytes(t[x + 2 : x + 4], "big")
#             angle = 360 - t[x + 4] / 256 * 360
#             min_quality = t[x + 5]
#             minutiaes.append(Minutiae(min_type, min_x, min_y, angle, min_quality))
#         return minutiaes

#     if format == "19794-2-2011":
#         with open(path, "rb") as f:
#             t = f.read()
#         magic = int.from_bytes(t[0:4], "big")
#         version = int.from_bytes(t[4:8], "big")
#         total_bytes = int.from_bytes(t[8:12], "big")
#         fp_count = int.from_bytes(t[12:14], "big")
#         HASCERTS = int.from_bytes(t[14:15], "big")
#         fp_bytes = int.from_bytes(t[15:19], "big")
#         year = int.from_bytes(t[19:21], "big")
#         month = int.from_bytes(t[21:22], "big")
#         day = int.from_bytes(t[22:23], "big")
#         hour = int.from_bytes(t[23:24], "big")
#         minute = int.from_bytes(t[24:25], "big")
#         second = int.from_bytes(t[25:26], "big")
#         millisecond = int.from_bytes(t[26:28], "big")
#         dev_tech = int.from_bytes(t[28:29], "big")
#         dev_vendor = int.from_bytes(t[29:31], "big")
#         dev_id = int.from_bytes(t[31:33], "big")
#         QCOUNT = int.from_bytes(t[33:34], "big")
#         FP_quality = int.from_bytes(t[34:35], "big")
#         Q_vendor = int.from_bytes(t[35:37], "big")
#         Q_algo = int.from_bytes(t[37:39], "big")
#         position = int.from_bytes(t[39:40], "big")
#         view_offset = int.from_bytes(t[40:41], "big")
#         resolution_x = int.from_bytes(t[41:43], "big")
#         resolution_y = int.from_bytes(t[43:45], "big")
#         sample_type = int.from_bytes(t[45:46], "big")
#         im_w = int.from_bytes(t[46:48], "big")
#         im_h = int.from_bytes(t[48:50], "big")
#         MINBYTES, ENDINGTYPE = (t[50] >> 4) & 0xF, t[50] & 0xF
#         minutiae_num = int.from_bytes(t[51:52], "big")
#         minutiaes = []
#         for i in range(minutiae_num):
#             x = 52 + 6 * i
#             min_type = (t[x] >> 6) & 0x3
#             min_x = int.from_bytes([t[x] & 0x3F, t[x + 1]], "big")
#             min_y = int.from_bytes(t[x + 2 : x + 4], "big")
#             angle = 360 - t[x + 4] / 256 * 360
#             min_quality = t[x + 5]
#             minutiaes.append(Minutiae(min_type, min_x, min_y, angle, min_quality))
#         return minutiaes


# def main():
#     minutiae_2005 = load_iso19794("isoTemplate", format="19794-2-2005")
#     print(minutiae_2005)

#     # minutiae_2011 = load_iso19794("iso2011template", format="19794-2-2011")
#     # print(minutiae_2011)


# if __name__ == "__main__":
#     main()


IsADirectoryError: [Errno 21] Is a directory: 'isoTemplate'